## SPAM HAM CLASSIFIER USING NAIVE BAYES AND PASSIVE AGGRESSIVE CLASSIFIER

In [31]:
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np

In [8]:
ps = PorterStemmer()
wnl = WordNetLemmatizer()

In [6]:
df = pd.read_csv("F:/files/smsspamcollection/SMSSpamCollection", sep = '\t', names = ['labels', 'messages'])

In [7]:
df

,labels,messages
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [11]:
for i in range(len(df['messages'])):
    clean = re.sub('[^a-zA-Z]', ' ', df['messages'][i])
    clean = clean.lower()
    words = nltk.word_tokenize(clean)
    words = [ps.stem(word) for word in words if word not in stopwords.words('english')]
    df['messages'][i] = ' '.join(words)

In [22]:
TfVector = TfidfVectorizer(max_features = 5000)

In [23]:
X = TfVector.fit_transform(df['messages'])

In [24]:
X = X.toarray()

In [25]:
Y = pd.get_dummies(df['labels'], drop_first = True)

In [26]:
X.shape

(5572, 5000)

In [27]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = 0.30)

In [30]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
model = MultinomialNB()

In [48]:
param = {'alpha' : np.linspace(1,11,10)}

In [49]:
Rcv = RandomizedSearchCV(model, param_distributions = param, n_iter = 100, cv = 5, verbose = 3, n_jobs = -1, return_train_score = True )

In [50]:
Rcv.fit(x_train,y_train)

C:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:278: UserWarning: The total space of parameters 10 is smaller than n_iter=100. Running 10 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    5.3s finished
C:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


RandomizedSearchCV(cv=5, estimator=MultinomialNB(), n_iter=100, n_jobs=-1,
                   param_distributions={'alpha': array([ 1.        ,  2.11111111,  3.22222222,  4.33333333,  5.44444444,
        6.55555556,  7.66666667,  8.77777778,  9.88888889, 11.        ])},
                   return_train_score=True, verbose=3)

In [72]:
model1 = MultinomialNB(alpha = 1.0)

In [73]:
model1.fit(x_train,y_train)

C:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


MultinomialNB()

In [74]:
y_pred = model1.predict(x_test)

In [75]:
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix

In [76]:
print(accuracy_score(y_pred,y_test))
print(classification_report(y_pred,y_test))
print(confusion_matrix(y_pred,y_test))

0.97188995215311
              precision    recall  f1-score   support

           0       1.00      0.97      0.98      1498
           1       0.79      1.00      0.88       174

    accuracy                           0.97      1672
   macro avg       0.89      0.98      0.93      1672
weighted avg       0.98      0.97      0.97      1672

[[1451   47]
 [   0  174]]


In [77]:
#in this case we need to focus on recall. because spam predicted as ham is not problem when considered to ham is predicted as spam.

In [79]:
y_test.value_counts()

spam
0       1451
1        221
dtype: int64

In [80]:
from sklearn.linear_model import PassiveAggressiveClassifier

In [81]:
PAC = PassiveAggressiveClassifier()

In [82]:
model3 = PAC.fit(x_train,y_train)

C:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [83]:
y_pred = model3.predict(x_test)

In [84]:
print(accuracy_score(y_pred,y_test))
print(classification_report(y_pred,y_test))
print(confusion_matrix(y_pred,y_test))

0.9802631578947368
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1472
           1       0.88      0.97      0.92       200

    accuracy                           0.98      1672
   macro avg       0.94      0.98      0.96      1672
weighted avg       0.98      0.98      0.98      1672

[[1445   27]
 [   6  194]]


In [85]:
#It gives more accuracy than that, recall and precision is also good.